In [1]:
import pandas as pd
import re, os
from openpyxl import workbook, load_workbook
from openpyxl.utils import get_column_letter
from collections import Counter


In [2]:
def voter(path):
    A,B,C,D,E,F,G,H = [],[],[],[],[],[],[],[]
    f_name = []
    House =[]
    Middle = []
    val, f = [],[]
    l, l2 = [], []
    p_no, section = [], []
    index = []
    index2 = []

    part_No = []
    Assembly = []
    pattern = re.compile(r'^Name')
    var = "Photo is"
    var2 = "Available"
    part = "None"

    wb = load_workbook(path)
    ws = wb.active
    #function for removing elements
    def rem(lst):
        if lst:
            for i in lst:
                f.remove(i)
                return f
        else:
            return f
    df = pd.read_excel(path)
    value=df.shape
    columns=value[-1]
    rows=value[0]
    df.head()
        
    pattern = re.compile(r'^Name:')
    ls = list(df.columns)

    for i,c in enumerate(ls):
        b=df[c].values.tolist()
        for k in b:
            m=str(k)
            if m.__contains__('Name'):
                index.append(i)
            if m.__contains__('Photo is'):
                 index2.append(i)
    
    col = Counter(index)
    index = [x for x in index if col[x] >=5]


    col2 = Counter(index2)
    index2 = [x for x in index2 if col2[x] >=5]
    
    res = [i for n, i in enumerate(index) if i not in index[:n]]
    res2 = [i for n, i in enumerate(index2) if i not in index2[:n]]

    for i in res:
        sum=i+1
        l.append(sum)

    for i in res2:
        sum=i+2
        l2.append(sum)
    for x,y in zip(l,l2):
        for row in range(1, rows):
            for col in range(x, y):
                char = get_column_letter(col)
                data=ws[char + str(row)].value
                val.append(data)
                f=[x for x in val if pd.isnull(x) == False]
                
    f = [i for i in  f if i != var]
    f = [i for i in  f if i != var2]
    val =2
    for i in range(len(f)):
        line=str(f[i])
        if line.startswith('Part number'):
            part = line
        
        if line.startswith('Assembly Constituency No and Name'):
            a_c=line
        
        if line.startswith('Section'):
            val +=1
            page_N = ('page {}'.format(val))
            sec = line
            
        name=re.search(pattern, line)
        if name:
            section.append(sec)
            p_no.append(page_N)
            
            
            A.append(f[i-2])
            B.append(f[i-1])
            C.append(f[i])
            Name=f[i+1]
            h=Name.find('Husband')
            p=Name.find('Father')
            o=Name.find('House')
            q=Name.find('Mother')
            z=Name.find('Other')
            if h and p and o and q and z:
                con = f[i] + " " + f[i+1]
                f_name.append(f[i+1])
                C.pop()
                C.append(con)
    f=rem(f_name)


    for j in range(len(f)):
        x=str(f[j])
        part_No.append(part)
        Assembly.append(a_c)
        name=re.search(pattern, x)
        if name:
            D.append(f[j+1])

            M_Name=f[j+2]
            #print(M_Name)
            if type(M_Name) != int:

                M_h=M_Name.find('Husband')
                M_p=M_Name.find('Father')
                M_o=M_Name.find('House')
                M_m=M_Name.find('Mother')
                if M_h and M_p and M_o and M_m:
                    con = f[j+1] + " " + f[j+2]
                    Middle.append(f[j+2])
                    D.pop()
                    D.append(con)

    f=rem(Middle)

    for j in range(len(f)):
        x=str(f[j])
        name=re.search(pattern, x)
        if name:
            E.append(f[j+2])

            M_Name=str(f[j+3])
            #print(M_Name)

            if type(M_Name) != float:
                M_h=M_Name.find('Age')
                M_p=M_Name.find('gAe')
                M_o=M_Name.find('House')
                M_m=M_Name.find('ge')
                M_n=M_Name.find('Ae')
                #print(type(M_Name))
                #print(E)
                if M_h and M_p and M_o and M_m and M_n:
                    con = str(f[j+2]) + " " + str(f[j+3])
                    House.append(f[j+3])
                    E.pop()
                    E.append(con)
            F.append(f[j+3])
            #G.append(f[j+4])
            #H.append(f[j+5])
    f=rem(House)
    First = pd.DataFrame(list(zip(Assembly,part_No, section, p_no, A, B, C, D, E, F)), columns =['Assembly','PartNo', 'section','Page', 'SR.NO','VoterID', 'Name', 'FHMO_Name','HOUSE NO','AGE'])
    #split  age and gender
    AGE = First['AGE'].str[:8]
    GENDER = First['AGE'].str[8:]
    First.drop(['AGE'], axis = 1)
    First['AGE'] = AGE
    First['GENDER'] = GENDER
    #First
    return First#.to_excel("DRY_RUN.xlsx")

In [3]:
Input = 'C:\\Arunachal PDFs\\Mother_integrated\\Done Files 16-20\\'
Output = 'C:\\Arunachal PDFs\\Mother_integrated\\Done Files 16-20\\Final excel\\'

In [4]:
EX, OUT = [], []
ex_li = os.listdir(Input)

df = pd.DataFrame()
for i in ex_li:
    if i.endswith('.xlsx'):
        print(Input+i)
        try:
            df1 = voter(Input+i)
            df = df.append(df1)
            df.to_excel(Output+i)
            OUT.append(Input+i)
            print(Output+i)
            
        except AttributeError:
            print("AttributeError")
            EX.append(Input+i)
            #print(Input+i)
        except UnboundLocalError:
            EX.append(Input+i)
        except TypeError:
            EX.append(Input+i)
        except IndexError:
            EX.append(Input+i)
            continue
        
        


C:\Arunachal PDFs\Mother_integrated\Done Files 16-20\S02A16P18.xlsx
AttributeError
C:\Arunachal PDFs\Mother_integrated\Done Files 16-20\S02A16P19.xlsx


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_8492\1199900709.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)


C:\Arunachal PDFs\Mother_integrated\Done Files 16-20\Final excel\S02A16P19.xlsx
C:\Arunachal PDFs\Mother_integrated\Done Files 16-20\S02A16P2.xlsx


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_8492\1199900709.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)


C:\Arunachal PDFs\Mother_integrated\Done Files 16-20\Final excel\S02A16P2.xlsx
C:\Arunachal PDFs\Mother_integrated\Done Files 16-20\S02A16P20.xlsx


...

C:\Arunachal PDFs\Mother_integrated\Done Files 16-20\Final excel\S02A20P7.xlsx
C:\Arunachal PDFs\Mother_integrated\Done Files 16-20\S02A20P8.xlsx


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_8492\1199900709.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)


C:\Arunachal PDFs\Mother_integrated\Done Files 16-20\Final excel\S02A20P8.xlsx
C:\Arunachal PDFs\Mother_integrated\Done Files 16-20\S02A20P9.xlsx


C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_8492\1199900709.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)


C:\Arunachal PDFs\Mother_integrated\Done Files 16-20\Final excel\S02A20P9.xlsx


In [5]:
Error_files = pd.DataFrame(list(zip(EX)), columns=['Error files'])
OUT_F = pd.DataFrame(list(zip(OUT)), columns=['Output files'])

In [9]:
print(Error_files)

                                          Error files
0   C:\Arunachal PDFs\Mother_integrated\Done Files...
1   C:\Arunachal PDFs\Mother_integrated\Done Files...
2   C:\Arunachal PDFs\Mother_integrated\Done Files...
3   C:\Arunachal PDFs\Mother_integrated\Done Files...
4   C:\Arunachal PDFs\Mother_integrated\Done Files...
5   C:\Arunachal PDFs\Mother_integrated\Done Files...
6   C:\Arunachal PDFs\Mother_integrated\Done Files...
7   C:\Arunachal PDFs\Mother_integrated\Done Files...
8   C:\Arunachal PDFs\Mother_integrated\Done Files...
9   C:\Arunachal PDFs\Mother_integrated\Done Files...
10  C:\Arunachal PDFs\Mother_integrated\Done Files...
11  C:\Arunachal PDFs\Mother_integrated\Done Files...
12  C:\Arunachal PDFs\Mother_integrated\Done Files...
13  C:\Arunachal PDFs\Mother_integrated\Done Files...


In [11]:
Error_files.to_excel('Error file list.xlsx')
OUT_F.to_excel('excell done file list.xlsx')